In [2]:
import numpy
import numpy as np
from numpy import trapz

import pandas as pd

import matplotlib

# Uses tinker as the backend for matplotlib (Not really sure what this is doing)
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

from pathlib import Path

import math
from math import pi
from math import sqrt

import statistics

import scipy.integrate
from scipy.signal import find_peaks
from scipy.signal import argrelmin
from scipy.integrate import cumtrapz

In [1]:
def calcCylinderSurfaceArea(radius, height):
    # Calculates the Surface area of a cylinder
    # Inputs:
        # radius: Radius of the cylinder
        # height of the cylinder

        baseArea = np.pi * radius**2 # Surface area of the base of the cylinder
        sideArea = 2 * np.pi * radius * height # Surface area on the side of the cylinder
        totalSurfaceArea = baseArea + sideArea
        return totalSurfaceArea

def calcConeLateralSurfaceArea(radius, height):
    # Calculates the lateral surface of a cone (Doesn't calc the surface area of the base of the cone)
    # Inputs:
        # radius: radius of the base of the cone
        # height: height of the cone (measured normal to the base)
        # sideSlope: Side slope of the cone

    return np.pi * radius * np.sqrt(height**2 + radius**2)

def calcCircleArea(radius):
    # Calc the area of a circle
    # Input:
        # radius: radius of the circle
    return np.pi * radius**2

def calcFFPConeContactArea(penetrationDepth, maxLength, baseRadius, coneTipRadius, coneTipHeight, areaCalcType):
    # Function calculates the contact area for the cone top of the FFP

    # penetrationDepth: Array of penetration depth values
    # maxLength: Max length before the lateral area of the FFP is considered
    # baseRadius: Radius of the Cone at the base (Circular side)
    # coneTipRadius: Radius of the cone tip (is not zero)
    # coneTipHeight: Height of the cone when measured normal to the circular base
    # areaCalcType: Selection of mantle or projected

    # Init array to store area
    area = np.ones(len(penetrationDepth))

    # Calc cone side slope
    coneSideSlope = (baseRadius-coneTipRadius)/coneTipHeight

    for i, depth in enumerate(penetrationDepth):
        
        # if the depth is greater than the maxLength of the cone
        if depth > maxLength:
            # decrease the depth used in the calculation to the maxLength
            depth = maxLength
        
        # Calc the radius
        radius = coneTipRadius + depth * coneSideSlope

        # Check area selection
        if areaCalcType == "mantle":
            # if selected calc mantle area (Same as surface area without the base of cone)
            area[i] = calcConeLateralSurfaceArea(radius, depth)

        elif areaCalcType == "projected":
            # if selected calc projected area
            area[i] = calcCircleArea(radius)

        else: 
            return ValueError("areaCalcType must be mantle or projected. Currently is: {}".format(areaCalcType))

    return area

def calcFFPBluntContactArea(penetrationDepth, maxLength, baseRadius, areaCalcType):

    # init array to store area calcs
    area = np.zeros(len(penetrationDepth))

    for i, depth in enumerate(penetrationDepth):
        # if the depth is greater than the maxLength of the blunt cylinder
        if depth > maxLength:
            # decrease the depth used in the calculation to the maxLength
            depth = maxLength
        
        # radius is constant for cylinder
        radius = baseRadius

        # Check area selection
        if areaCalcType == "mantle":
            # if selected calc mantle area (Surface Area of a cylinder)
            area[i] = calcCylinderSurfaceArea(radius, depth)

        elif areaCalcType == "projected":
            # if selected calc projected area
            area[i] = calcCircleArea(radius)

        else: 
            return ValueError("areaCalcType must be mantle or projected. Currently is: {}".format(areaCalcType))
        
    # Might need to convert this to m^2
    return area

def calcFFPContactArea(penetrationDepth,  maxLength, areaCalcType, tipType, baseRadius = 4.375, coneTipRadius = 0.22, coneTipHeight = 7.55,):
    # Function calls the required functions to calculate the generated area for a cone
    # Inputs:
        # penetrationDepth: Array of depths the FFP has penetrated, [m]
        # maxLength: max length of penetration before radius become constant, [cm]
        # baseRadius: Base radius of the cone and blunt tip, [cm]
        # coneTipRadius: Tip radius of the cone??, [cm]
        # coneTipHeight: height of the cone, [cm]
        # areaCalcType: "mantle or projected"
        # tipType: blunt or cone tip type       
    
    # Convert penetration depth to centimeters [cm]
    penetrationDepth = penetrationDepth * 100 #[cm]

    # Calc area for cone tip
    if tipType == "cone":
        area = calcFFPConeContactArea(penetrationDepth, maxLength, baseRadius, coneTipRadius, coneTipHeight, areaCalcType)

     # Calc area for blunt tip
    elif tipType == "blunt":
        area = calcFFPBluntContactArea(penetrationDepth, maxLength, baseRadius, areaCalcType)
    
    # Convert to m^2
    return area/1e4 #[m^2]
    
def calcDynamicBearingCapacity(acceleration, massFFP, contactArea, rho_w = 1020, volumeFFP = 0.002473, waterDrop = True ):
    # Function calculates the dynamic bearing capacity of the soil

    # Inputs
        # dropType (S: "air" or "water" drop
        # accleration: deaccleration array
        # massFFP: Mass of the Free Fall Penetrometer (FFP)
        # contactArea: Array of contact areas between the FFP and the soil
        # rho_w: Density of water
        # volumeFFP: Volume of the FFP

    if waterDrop:

        # Calc mass of water displaced by the FFP
        displacedWaterMass = rho_w * volumeFFP

        # Calc Buoyant mass of the FFP
        buoyantMass = massFFP - displacedWaterMass

        # Calc impact force
        force = buoyantMass * acceleration

    # Check for air drop
    elif not waterDrop:
        force = massFFP * acceleration

    # Dynamic bearing capacity
    qDyn = force/contactArea

    return qDyn

def calcQuasiStaticBearingCapacity(velocity, strainrateCorrectionType, qDyn, K_Factor = 0.1, refVelocity = 0.02, BrilliCoeff = 0.31):
    # Function calculates the quasi-static bearing capacity value
    # velocity: Array of FFP velocity values [m/s]
    # K_Factors: Array of K factors to be used

    # Local Variables: 
    # f_SR: Strain Rate factor
    # maxVelocity: Max velocity of the descent

    maxVelocity = np.max(velocity)

    if strainrateCorrectionType == "log":
        f_SR = 1 + K_Factor * np.log10(velocity/refVelocity)

    elif strainrateCorrectionType == "Brilli":
        # Strain rate correction factor following Brilli et al. (20??) Commonly used for air drops
        f_SR = 1 + BrilliCoeff * maxVelocity * np.log10(velocity/refVelocity)

    elif strainrateCorrectionType == "invHyperSin":
        # Inv hyperbolic sin correction factor following Stephan (2015) and Randolph (2004)
        f_SR = 1 + K_Factor/np.log(10) * np.arcsinh(velocity/refVelocity)

    else:
        ValueError("strainrateCorrectionType must be one of the folliwng: log, Brilli, invHyperSin.")

    # calc Quasi Static Bearing capacity value
    qsbc = qDyn/f_SR
    return qsbc

FFP_Properties =
    
def integrateFunc(x, func, order = 1):
    # x: Dependent Variable that the derivative should be taken with respect to
    # func: function that the derivative is being taken of
    # order: the order of the integral that should be taken

    pass

def calcDerivative(x, func, order = 1, type = "forward"):
    # x: Dependent Variable that the derivative should be taken with respect to
    # func: function that the derivative is being taken of
    # order: the order of the derivative that should be taken. 1 for first derivative, 2 for second...
    # Type of derivative that should be taken (forward, backwards, central)

    pass



    

SyntaxError: non-default argument follows default argument (1948369725.py, line 88)

In [3]:
100**2
1e4

10000.0

In [ ]:
def tipProperties(tipType):
    if tipType == "cone":
        mass = 7.71 # [kg]
        length = 8.48 - 0.93 #[cm], originally 7.87, 7.57 fors perfect 60 deg, 8.48 measured - .93 measured 90 deg
    if tipType == ""